In [1]:
import brightway2 as bw
import functools

## changelog
- previous version is alicia_import_20230317
- makes use of select_file.py to enable selection of .xlsx database through a GUI.
- makes use of db_dependencies.py to take an input of the names of the databases that are dependencies of the imported database.
Then the exchanges in the imported databases are linked with them, if they are already present in the brightway2 project.

In [2]:
from bw2io import *
from bw2io.strategies import *

In [3]:
bw.projects.set_current('alicia')

In [4]:
import importing.select_file_v2 as select
database_path, database_name = select.file_selection()
# for checking the paths
print(database_path)
print(database_name)

D:\Alicia\Work\Pycharm_projects\importing\databases\aircraftlca2.xlsx
aircraftlca2.xlsx


In [5]:
# Create an ExcelImporter object
ei = ExcelImporter(database_path)

Extracted 1 worksheets in 0.01 seconds


In [6]:
# apply the statistics and check for data statistics
ei.apply_strategies()
ei.statistics()

Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 11.75 seconds
2 datasets
13 exchanges
9 unlinked exchanges
  Type technosphere: 9 unique unlinked exchanges


(2, 13, 9)

In [7]:
ei.match_database(fields=('name')) #database self matching

import db_dependencies as DB
for i in range (DB.db_array_length):
    ei.match_database(db_name=DB.db_names[i], fields=('name', 'unit', 'location'))
#ei.match_database(None, ignore_categories=True)
ei.statistics()

Applying strategy: link_iterable_by_fields
Couldn't apply strategy link_iterable_by_fields:
	Object in source database can't be uniquely linked to target database.
Problematic dataset is:
{'a': '(missing)',
 'e': '(missing)',
 'filename': '(missing)',
 'm': '(missing)',
 'n': '(missing)'}
Possible targets include (at least one not shown):
[{'a': '(missing)',
  'e': '(missing)',
  'filename': '(missing)',
  'm': '(missing)',
  'n': '(missing)'}]
Applying strategy: link_iterable_by_fields
Couldn't apply strategy link_iterable_by_fields:
	Object in source database can't be uniquely linked to target database.
Problematic dataset is:
{'filename': '(missing)',
 'location': 'RoW',
 'name': 'treatment of metal scrap, mixed, for recycling, unsorted, sorting',
 'unit': 'kilogram'}
Possible targets include (at least one not shown):
[{'filename': '5c948344-e140-5d1b-91ee-65c515f1d571_858164b0-44f1-4118-b6cb-47054faa21ef.spold',
  'location': 'RoW',
  'name': 'treatment of metal scrap, mixed, for r

(2, 13, 4)

In [ ]:
# To "neatly" print the entire imported data within ei. ONLY use when requried
import pprint
pprint.pprint(ei.data())

In [8]:
# if there are any unlinked exchanges, the databases of these exchanges, and the unlinked exchanges are stored in their own arrays
unlinked_databases = []
unlinked_exchanges = []
for exc in ei.unlinked:
    if exc["database"] not in unlinked_databases:
        unlinked_databases.append(exc["database"])
    unlinked_exchanges.append(exc)
# the unlinked exchanges are stored as a list, where each element is a dict
print ("The unlinked database/s:\n",unlinked_databases)
print("The unlinked exchanges are:\n",unlinked_exchanges)

The unlinked database/s:
 ['cutoff391']
The unlinked exchanges are:
 [{'name': 'treatment of metal scrap, mixed, for recycling, unsorted, sorting', 'amount': 0, 'database': 'cutoff391', 'location': 'RoW', 'unit': 'kilogram', 'type': 'technosphere', 'formula': '6407.56 * Al_select_for_fuselage', 'original_amount': 1, 'reference product': 'aluminium scrap, post-consumer, prepared for melting'}, {'name': 'electricity, high voltage, European attribute mix', 'amount': 8.33, 'database': 'cutoff391', 'location': 'RER', 'unit': 'kilowatt hour', 'type': 'technosphere', 'comment': 'energy requied for pyrolysis is 30 MJ/kg of CFRP', 'reference product': 'electricity, high voltage'}, {'name': 'market for transport, freight, lorry >32 metric ton, EURO6', 'amount': 2364.415, 'database': 'cutoff391', 'location': 'RER', 'unit': 'ton kilometer', 'type': 'technosphere', 'formula': 'vol_fuselage * CFRP_select_for_fuselage * CFRP_density/1000000*500', 'comment': 'assuming a 500 km transport for each ton',

If there are unlinked technosphere flows (from ecoinvent), then there are ##two possible methods## to resolve this:
1. first store the matched database to a separate excel file. Then, perform a self link, and write the database. Afterwards, these exchanges can be manually verified through ActivityBrowser and reconnected with ecoinvent.
2. for these specific unlinked exchanges, change the name of the database to "self". That is, the database name is self referenced. This is faster. However, this method results in the loss of the linking with ecoinvent.
*note*: only enable the "nuclear" option, if nothing else is working. If so, please compare the "matched database" with the database in activity browser to see which all have been dropped. Add them again.

In [11]:
#write an excel file with all the matched activities so far.copy the excel file that was written in the previous line to your desired location for further reference.
ei.write_excel(only_unlinked=False)
ei.add_unlinked_activities()
ei.statistics()
#uncomment the line below to enable the Nuclear option
# ei.drop_unlinked(i_am_reckless=True)

Wrote matching file to:
C:\Users\nair_ra\AppData\Local\pylca\Brightway3\alicia.e94ef563867e9c9df3fcc999bdb045f5\output\db-matching-aircraftlca2.xlsx


NameError: name 'aircraftlca2' is not defined

In [10]:
ei.statistics()
#ei.add_unlinked_activities()

2 datasets
13 exchanges
4 unlinked exchanges
  Type technosphere: 4 unique unlinked exchanges


(2, 13, 4)

In [ ]:
#run this block if there any unlinked exchanges in In 7.
print(unlinked_databases)
for i in range(len(unlinked_databases)):
    ei.match_database(unlinked_databases[i], fields=('name'))
#ei.add_unlinked_activities()
#ei.match_database("materials") #since there are exchanges linking to this database

In [ ]:
#check for statistics again before writing database
ei.statistics()

In [ ]:
# will write to the database only if there are no unlinked exchanges
#writes the database into the bw2 as the same name of the excel sheet
ei.write_database()

In [ ]:
quit()